In [5]:
import dotenv
dotenv.load_dotenv()

True

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

In [6]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

In [10]:
# parquet files generated from indexing pipeline
INPUT_DIR = "./output/20240704-221917/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [11]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 39


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,38,# Scrooge's Transformation and the Spirit of C...,2,8.5,Scrooge's Transformation and the Spirit of Chr...,The impact severity rating is high due to the ...,This report delves into the pivotal characters...,"[{'explanation': 'The Spirit, identified as th...","{\n ""title"": ""Scrooge's Transformation and ...",37dd346a-4c36-43ea-aef6-7a248cc63a55
1,37,# The Transformation of Ebenezer Scrooge: A St...,2,8.5,The Transformation of Ebenezer Scrooge: A Stud...,The impact severity rating is high due to the ...,This report delves into the intricate network ...,[{'explanation': 'Ebenezer Scrooge's journey b...,"{\n ""title"": ""The Transformation of Ebeneze...",86662713-973c-4144-b045-ee2183c87978
2,15,"# Scrooge, The Clerk, and Christmas-time: A St...",1,7.5,"Scrooge, The Clerk, and Christmas-time: A Stud...",The impact severity rating is relatively high ...,This report delves into the intricate relation...,[{'explanation': 'Scrooge and The Clerk share ...,"{\n ""title"": ""Scrooge, The Clerk, and Chris...",5020dc21-78db-4c9e-baa3-709d909769c7
3,16,# Spectral Guidance and Redemption in A Christ...,1,8.5,Spectral Guidance and Redemption in A Christma...,The high impact severity rating reflects the p...,This report explores the interconnected roles ...,[{'explanation': 'The Ghost is central to 'A C...,"{\n ""title"": ""Spectral Guidance and Redempt...",0edb24dc-71f3-4d72-b2b4-2772ae1771d6
4,17,# Scrooge's Counting-House and the Spirit of C...,1,4.0,Scrooge's Counting-House and the Spirit of Chr...,"The impact severity rating is moderate, reflec...",This report delves into the dynamics within Sc...,[{'explanation': 'Scrooge exhibits a clear aut...,"{\n ""title"": ""Scrooge's Counting-House and ...",cdbb296f-b83b-4845-8a5f-a618a8bd9336


In [12]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [13]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [14]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [15]:
result = await search_engine.asearch(
    "What is the major conflict in this story and who are the protagonist and antagonist?"
)

print(result.response)

### Major Conflict and Characters in "A Christmas Carol"

#### The Major Conflict

The central conflict in "A Christmas Carol" is the profound transformation of Ebenezer Scrooge from a miserly, cold-hearted man into a figure of generosity and warmth. This internal struggle is sparked by his disdain for Christmas and humanity, which is radically altered through his moral and spiritual awakening. The intervention of various spirits underscores themes of redemption, compassion, and the impact of one's actions on others, highlighting the story's deep exploration of personal change and societal implications [Data: Reports (37, 23, 21, 18, +more)].

#### The Protagonist

Ebenezer Scrooge is the protagonist whose journey from greed and isolation to generosity and community involvement forms the narrative's core. His character embodies the potential for personal change and redemption, serving as the focal point of the story's major conflict. Scrooge's transformation is not just a personal vict

In [16]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,23,The Cratchit Family: A Tale of Hope and Resili...,0.194631,# The Cratchit Family: A Tale of Hope and Resi...,7.5
1,21,The Ghost's Guiding Influence in Scrooge's Tra...,0.140940,# The Ghost's Guiding Influence in Scrooge's T...,8.5
2,24,The Cratchit Family Christmas,0.140940,# The Cratchit Family Christmas\n\nThe Cratchi...,3.5
3,27,Scrooge's Family Christmas Dynamics,0.127517,# Scrooge's Family Christmas Dynamics\n\nThis ...,6.5
4,18,The Spirit's Influence in A Christmas Carol,0.114094,# The Spirit's Influence in A Christmas Carol\...,8.5
5,28,The Spirit of Christmas and Its Impact on Char...,0.093960,# The Spirit of Christmas and Its Impact on Ch...,8.5
6,34,Fezziwig's Christmas Eve Celebration,0.093960,# Fezziwig's Christmas Eve Celebration\n\nThis...,7.5
7,26,The Scrooge-Fred Family Dynamics and Supernatu...,0.053691,# The Scrooge-Fred Family Dynamics and Superna...,7.5
8,32,The Parlour Community: A Study of Morality and...,0.053691,# The Parlour Community: A Study of Morality a...,6.5
9,30,Joe's Network of Transactions and Interactions,0.046980,# Joe's Network of Transactions and Interactio...,4.0


In [17]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 3. LLM tokens: 23811
